In [2]:
import os
import sys
import json
import yaml

In [8]:
with open("./config.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [11]:
with open(
        os.path.join(
            config["checkpoints"][0]["checkpoint_path"],
            "config.json"
            )
    ) as model_config:
    model_config = json.load(model_config)

In [72]:
byoc_params = {
    "model_arch":model_config["model_type"],
    "seq_length":model_config["max_position_embeddings"],
    "vocab_size":model_config["vocab_size"],
    "param_count":8
}
byoc_params

#"param_count"?

{'model_arch': 'llama',
 'seq_length': 8192,
 'vocab_size': 128256,
 'param_count': 8}

In [73]:
from src.snsdk_byoc_wrapper import BYOC
byoc = BYOC("./config.yaml")

2024-11-01 14:38:48,419 [INFO] Using config file located in ./config.yaml
2024-11-01 14:38:48,421 [INFO] Using variables from Snapi config to set up Snsdk.


In [74]:
sstudio_models=byoc.list_models(verbose=True)

In [76]:
for i in range(len(sstudio_models)):
    print(i, sstudio_models[i]["architecture"])

0 GPT 1.5B
1 
2 
3 
4 GPT 1.5B
5 LLaMA v2 7B
6 Transformers
7 Neural Diarization
8 Transformers
9 BERT base
10 Transformers
11 Transformers
12 Transformers
13 HuBERT
14 GPT 13B
15 GPT 13B
16 BERT
17 Transformers
18 Llama v3
19 LLaMA v2 70B
20 BERT
21 Composite
22 
23 GPT 13B
24 BERT
25 Composite
26 BERT base
27 LLaVA
28 Composite
29 
30 Transformers
31 CLIP ViT B
32 GPT 1.5B
33 Llama v2
34 Deepseek 6.7B
35 CLIP ViT B
36 Llama v2
37 Llama 70B
38 Falcon 40B
39 GPT 1.5B
40 Composite
41 Composite
42 Composite
43 BERT
44 Transformers
45 Composite
46 
47 BERT
48 
49 CLIP ViT B
50 Composite
51 GPT 13B
52 GPT 1.5B
53 Transformers
54 Transformers
55 Composite
56 Transformers
57 
58 Transformers
59 
60 BERT
61 Composite
62 
63 DePlot
64 
65 Transformers
66 Transformers
67 Mistral
68 Transformers
69 
70 GPT 13B
71 GPT 13B
72 GPT 13B
73 
74 
75 LLaMA v2 7B
76 LLaMA v2 7B
77 LLaMA v2 70B
78 Llama v3
79 LLaMA v2 7B
80 
81 Transformers
82 Llama v3
83 Transformers
84 LLaMA v2 7B
85 
86 LLaMA v2 7B
87 

In [89]:
suitable_apps = []

for model in sstudio_models:
    params = model.get("params",{})
    if params is not None:
        model_params=params.get("invalidates_checkpoint")
        if model_params is not None:
            #if byoc_params["model_arch"].lower() in model["architecture"].lower():
                if str(byoc_params["param_count"])+'b' == model_params.get("model_parameter_count"):
                    if byoc_params["vocab_size"] == model_params.get("vocab_size"):
                        if byoc_params["seq_length"] == model_params.get("max_seq_length"):
                            suitable_apps.append(model["app_id"])
        
suitable_apps=set(suitable_apps)
suitable_apps

{'61fa0993-04a2-42ca-9db1-1eff693ea978',
 'ad39e323-9878-4914-8e29-82c9f2939475'}

In [92]:
app_list = byoc.list_apps()

for app in app_list:
    if app["id"] in suitable_apps:
        print(app)

{'id': '61fa0993-04a2-42ca-9db1-1eff693ea978', 'name': 'Samba1 Llama3 Experts'}
{'id': 'ad39e323-9878-4914-8e29-82c9f2939475', 'name': 'Llama 3'}
